# CS 690 Homework1

## Overview
        
In this homework, I read in two datasets from Gapminder, which are **CO2 emissions (tonnes per person)** and **Income per person (GDP/capita, PPP$ inflation-adjusted)**. Then I used two datasets from the sample data of Bokeh to help me make more meaningful visulizations. The two datasets are **gapminder_regions** and **gapminder_population**. 

## Plots Overview

There are three plots in this homework, a bubble chart, a line chart, and a bar chart. All of these visualizations contain **box zoom**, **wheel zoom**, and **pan**. In particular, the bubble chart is implemented with a **slider** to choose the data from a specific year and **hovers** to show the country name of each country, the line chart is implemented with **legend-muting** characteristic. 

In [137]:
import numpy as np
from pandas import DataFrame as df

def process_data_bubble():
    gdp = df.from_csv('gdp1.csv')
    co2 = df.from_csv('co2_person.csv')
    population = df.from_csv('population1.csv')
    regions = df.from_csv('gapminder_regions.csv')

    # Make the column names ints not strings for handling
    columns = list(co2.columns)
    index = -1
    for col in columns:
        index += 1
        if int(col) == 1800:
            break
    end_index = -1
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(co2.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1
    columns_new = columns[20 :60]
    index_end = 200
    co2 = co2.loc[indexes[0]:indexes[index_end], columns_new]
    years = [int(i) for i in columns_new]
    rename_dict = dict(zip(columns_new, years))

    end_index = -1
    columns = list(gdp.columns)
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(gdp.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1
    gdp = gdp.loc[indexes[0]: indexes[index_end], columns_new]
    end_index = -1
    columns = list(population.columns)
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(population.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1
    population = population.loc[indexes[0]: indexes[index_end], columns_new]

    co2 = co2.rename(columns = rename_dict)
    gdp = gdp.rename(columns = rename_dict)
    gdp = gdp / 800
    population = population.rename(columns = rename_dict)
    regions = regions.rename(columns = rename_dict)

    regions_list = list(regions.Group.unique())

    # Turn population into bubble sizes. Use min_size and factor to tweak.
    scale_factor = 100
    population_size = np.sqrt(population / (np.pi)) / scale_factor
    min_size = 3
    population_size = population_size.where(population_size >= min_size).fillna(min_size)


    return co2, gdp, population_size, regions, years, regions_list

In [138]:
# -*- coding: utf-8 -*-
import pandas as pd

from bokeh.core.properties import field
from ipywidgets import interact
from bokeh.io import push_notebook, output_notebook
from bokeh.layouts import layout
from bokeh.models import (
    ColumnDataSource, HoverTool, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper, CustomJS,
)
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, show
from bokeh.layouts import column, widgetbox
output_notebook()

Loading BokehJS ...

In [139]:
co2_df, gdp_df, population_size, regions_df, years, regions_list = process_data_bubble()

p = pd.Panel({'co2': co2_df, 'gdp': gdp_df, 'population': population_size})

data = {}

region_name = regions_df.Group
region_name.name = 'region'

for year in years:
    df = pd.concat([p.loc[:, :, year], region_name], axis=1).reset_index()
    data[year] = df.to_dict('series')

source = ColumnDataSource(data=data[years[0]])

plot = figure(x_range=(-0.2, 40), y_range=(0, 30), title='CO2 per capita vs. GDP per person by PPP', plot_height=600, plot_width= 750)
plot.xaxis.ticker = SingleIntervalTicker(interval=1)
plot.xaxis.axis_label = "GDP per person by PPP*"
plot.yaxis.ticker = SingleIntervalTicker(interval=2)
plot.yaxis.axis_label = "CO2 per capita"

label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='#eeeeee')
plot.add_layout(label)

color_mapper = CategoricalColorMapper(palette=Spectral6, factors=regions_list)
r = plot.circle(
    x='gdp',
    y='co2',
    size='population',
    source=source,
    fill_color={'field': 'region', 'transform': color_mapper},
    fill_alpha=0.8,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend=field('region'),
)

plot.add_tools(HoverTool(tooltips="@index", show_arrow=True, point_policy='follow_mouse'))

def update_notebook(y=0):
    source = ColumnDataSource(data=data[years[int(y)]])
    r.data_source.data = source.data
    label.text = str(years[int(y)])
    push_notebook()


### Bubble Chart

Below is a bubble chart depicting the relationship between CO2 emission per capita and GDP per person by PPP. Countries in different areas are in different colors. The size of the bubbles reflects the population size of that country. The hover will tell you the country name of each bubble if you put your mouse on the bubble. You can also box zoom and wheel zoom. There is a slider behind this plot, and we can visualize the data in different years by controlling the slider. It seems that countries with higher GDP per person have higher CO2 emission per capita. 

(*This plot may take a while to load on nbviewer*)

In [140]:
show(plot, notebook_handle = True)

In [100]:
interact(update_notebook, y = (0,len(years)))

In [141]:
import numpy as np
from pandas import DataFrame as df

def process_data_line():
    co2 = df.from_csv('co2_person.csv')

    # Make the column names ints not strings for handling
    columns = list(co2.columns)
    index = -1
    for col in columns:
        index += 1
        if int(col) == 1800:
            break
    end_index = -1
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(co2.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1

    columns_new = columns[0 :end_index]
    index_end = 231
    co2 = co2.loc[indexes[0]:indexes[index_end], columns_new]
    years = [int(i) for i in columns_new]
    rename_dict = dict(zip(columns_new, years))
    
    co2 = co2.rename(columns = rename_dict)

    china_df = co2.loc['China', :].values.tolist()
    us_df = co2.loc['United States', :].values.tolist()
    mx_df = co2.loc['Mexico', :].values.tolist()
    
    return china_df, us_df, mx_df, years


In [147]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import SingleIntervalTicker
from bokeh.io import output_notebook
output_notebook()

china_df, us_df, mx_df, years = process_data_line()

plot = figure(title='CO2 per capita vs. Year', plot_height=600, plot_width= 750)
plot.xaxis.ticker = SingleIntervalTicker(interval=20)
plot.xaxis.axis_label = "Year"
plot.yaxis.ticker = SingleIntervalTicker(interval=2)
plot.yaxis.axis_label = "CO2 per capita"
years_new = []
df_group = [china_df, us_df, mx_df]
color_group = [['red', 'China'], ['blue', 'United States'], ['green', 'Mexico']]

for values, cl in zip(df_group, color_group):
    plot.line(x = years, y = values, color = cl[0], alpha=0.8, muted_color=cl[0], muted_alpha=0.2, legend = cl[1])

plot.legend.location = 'top_left'
plot.legend.click_policy="mute"


Loading BokehJS ...

### Line Chart

The line chart shows the variation curve of CO2 per capita of three representative countries. United States is a deveploed country, China is a BRIC developing country, and Mexico is a non-BRIC developing country. If you click on the country name in the legend area, the line of the selected country will be muted.

(This plot may take a while to load on nbviewer)

In [148]:
show(plot, notebook_handle = True)

In [144]:
import numpy as np
from pandas import DataFrame as df

def process_data_bar():
    co2 = df.from_csv('co2_person.csv')
    population = df.from_csv('population1.csv')
    regions = df.from_csv('gapminder_regions.csv')
    columns = list(co2.columns)
    index = -1
    for col in columns:
        index += 1
        if int(col) == 1800:
            break
    end_index = -1
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(co2.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1
    columns_new = columns[0 :end_index]
    index_end = 231
    co2 = co2.loc[indexes[0]:indexes[index_end], columns_new]
    years = [int(i) for i in columns_new]
    rename_dict = dict(zip(columns_new, years))
    end_index = -1
    columns = list(population.columns)
    for col in columns:
        try:
            col = int(col)
            end_index += 1
        except:
            pass
    end_row = -1
    indexes = list(population.index)
    for row in indexes:
        if row is not None:
            if isinstance(row, str):
                end_row += 1
    population = population.loc[indexes[0]: indexes[index_end], columns_new]

    co2 = co2.rename(columns = rename_dict)
    population = population.rename(columns = rename_dict)
    regions = regions.rename(columns = rename_dict)
    regions_list = list(regions.Group.unique())

    co2 = co2.loc[:, columns_new[-5]:columns_new[-4]]
    population = population.loc[:, columns_new[-5]:columns_new[-4]]
    population = population / 100000

    return co2, population, regions, regions_list

In [149]:
import numpy as np
import pandas as pd
from bokeh.charts import Bar, show
from bokeh.layouts import row
from bokeh.io import output_notebook
output_notebook()

co2_per, population, regions, regions_list = process_data_bar()

co2_dict = dict()
for i in list(co2_per.index)[0:250]:
    if i in list(population.index) and i in regions.Group:
        try:
            sums = int(co2_per.loc[i, :].values.tolist()[0] * population.loc[i, :].values.tolist()[0])
            area = regions.Group[i]
            if area in co2_dict:
                co2_dict[area] += sums
            else:
                co2_dict[area] = sums
        except:
            pass

co2_values = []
co2_keys = []
for k in co2_dict:
    co2_keys.append(k)
    co2_values.append(co2_dict[k])

data = {
    'area' : co2_keys,
    'CO2 emission' : co2_values
}

bar2 = Bar(data, values='CO2 emission', label=['area'],
           agg='sum', title="CO2 total emission vs. Area", plot_width=750, legend='top_right')


Loading BokehJS ...

### Bar Chart

The bar chart shows the relationship between CO2 total emission and area. 

In [150]:
show(bar2, notebook_handle = True)

E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='01c82c2f-013f-4f3c-81ae-993bd1d66458', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='9b735937-011e-4e1d-ae28-23b0b03278c9', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='a243322e-69cb-4f17-bb13-dbd2942e09e8', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='a2860ffc-d6de-4811-b0cd-1f387a90bad5', ...)
E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='bca5703a-78c6-4a0f-acd9-eb3b87e51c41', ...)
E-1010 (CDSVIEW